

![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://githubtocolab.com/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/streamlit_notebooks/SENTIMENT_EN_SARCASM.ipynb)




# **Detect Sarcasm in text**

## 1. Colab Setup

In [ ]:
# Install PySpark and Spark NLP
! pip install -q pyspark==3.3.0 spark-nlp==4.2.8

In [12]:
import pandas as pd
import numpy as np
import json

import sparknlp
import pyspark.sql.functions as F

from sparknlp.annotator import *
from sparknlp.base import *
from pyspark.ml import Pipeline
from pyspark.sql import SparkSession
from sparknlp.pretrained import PretrainedPipeline
from pyspark.sql.types import StringType, IntegerType

## 2. Start Spark Session

In [11]:
spark = sparknlp.start()

print("Spark NLP version", sparknlp.version())
print("Apache Spark version:", spark.version)

spark

Spark NLP version 4.2.8
Apache Spark version: 3.3.0


## 3. Select the DL model

In [4]:
MODEL_NAME='classifierdl_use_sarcasm'

## 4. Some sample examples

In [5]:
## Generating Example Files ##
text_list = [
             #sarcasm
             """Love getting home from work knowing that in less than 8hours you're getting up to go back there again.""",
             #neutral
             """Oh my gosh! Can you imagine @JessieJ playing piano on her tour while singing a song. I would die and go to heaven. #sheisanangel""",
             #sarcasm
            """Dear Teva, thank you for waking me up every few hours by howling. Your just trying to be mother natures alarm clock.""",
             #neutral
             """The United States is a signatory to this international convention""",
             #sarcasm
             """If I could put into words how much I love waking up at am on Tuesdays I would""",
             #neutral
             """@pdomo Don't forget that Nick Foles is also the new Tom Brady. What a preseason! #toomanystudQBs #thankgodwedonthavetebow""",
             #sarcasm
             """I cant even describe how excited I am to go cook noodles for hours""",
             #neutral
             """@Will_Piper should move back up fella. I'm already here... On my own... Having loads of fun""",
             #sarcasm
             """Tweeting at work... Having sooooo much fun and honestly not bored at all #countdowntillfinish""",
             #neutral
             """I can do what I want to. I play by my own rules""",
             ]

## 5. Define Spark NLP pipleline

In [6]:
documentAssembler = DocumentAssembler()\
  .setInputCol("text")\
  .setOutputCol("document")
    
use = UniversalSentenceEncoder.pretrained(name="tfhub_use", lang="en")\
 .setInputCols(["document"])\
 .setOutputCol("sentence_embeddings")

sentimentdl = ClassifierDLModel.pretrained(name=MODEL_NAME)\
  .setInputCols(["sentence_embeddings"])\
  .setOutputCol("sentiment")

nlpPipeline = Pipeline(stages = [
                                documentAssembler,
                                use,
                                sentimentdl])


tfhub_use download started this may take some time.
Approximate size to download 923.7 MB
[OK!]
classifierdl_use_sarcasm download started this may take some time.
Approximate size to download 21.3 MB
[OK!]


## 6. Run the pipeline

In [7]:
df = spark.createDataFrame(text_list, StringType()).toDF("text")
result=nlpPipeline.fit(df).transform(df)

## 7. Visualize results

In [8]:
result.select(F.explode(F.arrays_zip(result.document.result, 
                                     result.sentiment.result)).alias("cols")) \
      .select(F.expr("cols['0']").alias("document"),
              F.expr("cols['1']").alias("sentiment")).show(truncate=False)

+--------------------------------------------------------------------------------------------------------------------------------+---------+
|document                                                                                                                        |sentiment|
+--------------------------------------------------------------------------------------------------------------------------------+---------+
|Love getting home from work knowing that in less than 8hours you're getting up to go back there again.                          |sarcasm  |
|Oh my gosh! Can you imagine @JessieJ playing piano on her tour while singing a song. I would die and go to heaven. #sheisanangel|normal   |
|Dear Teva, thank you for waking me up every few hours by howling. Your just trying to be mother natures alarm clock.            |sarcasm  |
|The United States is a signatory to this international convention                                                               |normal   |
|If I could p